### Ingestion de la carpeta "production_company"

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-30')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando DataFrameReader de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
productions_countries_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
])

In [0]:
productions_countries_df = spark.read \
    .schema(productions_countries_schema) \
    .option('multiline', True) \
    .json(f'{bronze_folder_path}/{v_file_date}/production_country')

### Paso 2 - Renombrar y Agregar Columnas
1. "movieId" renombrar a "movie_id"
2. "countryId" renombrar a "country_id"
3. Agregar columna "ingestion_date"
4. Agregar columna "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_countries_final_df = add_ingestion_date(productions_countries_df) \
    .withColumnRenamed('movieId', 'movie_id') \
    .withColumnRenamed('countryId', 'country_id') \
    .withColumn('enviroment', lit(v_enviroment)) \
    .withColumn('file_date', lit(v_file_date))

### Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(productions_countries_final_df, 'movie_silver','productions_countries', 'file_date')

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date  = src.file_date'
merge_delta_lake(productions_countries_final_df, "movie_silver", "productions_countries", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.productions_countries
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f'{silver_folder_path}/productions_countries'))

In [0]:
dbutils.notebook.exit("Success")